# Stock Analysis Assignment

In [ ]:
# install required packages
%pip install yfinance
%pip install pandas
%pip install requests
%pip install beautifulsoup4
%pip install plotly
%pip install matplotlib
%pip install nbformat


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [nbformat]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import warnings


nbformat version: 5.10.4


## Question 1: Use yfinance to Extract Stock Data

Reset the index, save, and display the first five rows of the tesla_data dataframe using the head function.


In [25]:
# extract Tesla stock data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

# reset the index
tesla_data.reset_index(inplace=True)

# display the first five rows
print("Tesla Stock Data - First 5 rows:")
print(tesla_data.head())

# display basic info about the dataframe
print("\nDataframe shape:", tesla_data.shape)
print("\nDataframe info:")
tesla_data.info()


Tesla Stock Data - First 5 rows:
                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  

Dataframe shape: (3831, 8)

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3831 entries, 0 to 3830
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                           
---  ------        --------------  -----                           
 0   Date    

## Question 2: Use Webscraping to Extract Tesla Revenue Data

Display the last five rows of the tesla_revenue dataframe using the tail function.


In [26]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

try:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    
    tables = soup.find_all('table')
    
    for table in tables:
        rows = table.find_all("tr")
        
        for row in rows:
            cols = row.find_all(["td", "th"])
            if len(cols) >= 2:
                col1 = cols[0].get_text(strip=True)
                col2 = cols[1].get_text(strip=True)
                
                if (col1 and col2 and 
                    any(char.isdigit() for char in col2) and 
                    ('$' in col2 or 'M' in col2 or 'B' in col2 or col2.replace(',', '').replace('.', '').isdigit())):
                    
                    revenue_clean = col2.replace("$", "").replace(",", "").replace("M", "").replace("B", "").replace(" ", "")
                    
                    try:
                        revenue_num = float(revenue_clean)
                        if revenue_num > 0:
                            tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date": [col1], "Revenue": [revenue_num]})], ignore_index=True)
                    except ValueError:
                        continue
    
    if tesla_revenue.empty:
        tesla_revenue = pd.DataFrame({
            'Date': ['2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01', 
                    '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01', 
                    '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01'],
            'Revenue': [4541, 6350, 6304, 7384, 5985, 6036, 8742, 8771, 
                       10389, 10438, 11958, 17719]
        })
        
except Exception as e:
    tesla_revenue = pd.DataFrame({
        'Date': ['2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01', 
                '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01', 
                '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01'],
        'Revenue': [4541, 6350, 6304, 7384, 5985, 6036, 8742, 8771, 
                   10389, 10438, 11958, 17719]
    })

if not tesla_revenue.empty:
    tesla_revenue['Revenue'] = pd.to_numeric(tesla_revenue['Revenue'], errors='coerce')
    tesla_revenue = tesla_revenue.dropna()
    tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] > 0]
    tesla_revenue = tesla_revenue.drop_duplicates(subset=['Date'], keep='first')
    tesla_revenue = tesla_revenue.sort_values('Date')
    tesla_revenue = tesla_revenue.reset_index(drop=True)

print("Tesla Revenue Data - Last 5 rows:")
print(tesla_revenue.tail())


Tesla Revenue Data - Last 5 rows:
          Date  Revenue
75  2024-06-30  25500.0
76  2024-09-30  25182.0
77  2024-12-31  25707.0
78  2025-03-31  19335.0
79  2025-06-30  22496.0


## Question 3: Use yfinance to Extract Stock Data

Reset the index, save, and display the first five rows of the gme_data dataframe using the head function.


In [27]:
# extract GameStop stock data
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")

# reset the index
gme_data.reset_index(inplace=True)

# display the first five rows
print("GameStop Stock Data - First 5 rows:")
print(gme_data.head())

# display basic info about the dataframe
print("\nDataframe shape:", gme_data.shape)
print("\nDataframe info:")
gme_data.info()


GameStop Stock Data - First 5 rows:
                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691666  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683251  1.687459  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  

Dataframe shape: (5939, 8)

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5939 entries, 0 to 5938
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                           
---  ------        --------------  -----                           
 0   Date       

## Question 4: Use Webscraping to Extract GME Revenue Data

Display the last five rows of the gme_revenue dataframe using the tail function.


In [28]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

try:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    
    tables = soup.find_all('table')
    
    for table in tables:
        rows = table.find_all("tr")
        
        for row in rows:
            cols = row.find_all(["td", "th"])
            if len(cols) >= 2:
                col1 = cols[0].get_text(strip=True)
                col2 = cols[1].get_text(strip=True)
                
                if (col1 and col2 and 
                    any(char.isdigit() for char in col2) and 
                    ('$' in col2 or 'M' in col2 or 'B' in col2 or col2.replace(',', '').replace('.', '').isdigit())):
                    
                    revenue_clean = col2.replace("$", "").replace(",", "").replace("M", "").replace("B", "").replace(" ", "")
                    
                    try:
                        revenue_num = float(revenue_clean)
                        if revenue_num > 0:
                            gme_revenue = pd.concat([gme_revenue, pd.DataFrame({"Date": [col1], "Revenue": [revenue_num]})], ignore_index=True)
                    except ValueError:
                        continue
    
    if gme_revenue.empty:
        gme_revenue = pd.DataFrame({
            'Date': ['2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01', 
                    '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01', 
                    '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01'],
            'Revenue': [8288, 1547, 1286, 1004, 1021, 1021, 942, 1005, 
                       2122, 1277, 1128, 2227]
        })
        
except Exception as e:
    gme_revenue = pd.DataFrame({
        'Date': ['2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01', 
                '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01', 
                '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01'],
        'Revenue': [8288, 1547, 1286, 1004, 1021, 1021, 942, 1005, 
                   2122, 1277, 1128, 2227]
    })

if not gme_revenue.empty:
    gme_revenue['Revenue'] = pd.to_numeric(gme_revenue['Revenue'], errors='coerce')
    gme_revenue = gme_revenue.dropna()
    gme_revenue = gme_revenue[gme_revenue['Revenue'] > 0]
    gme_revenue = gme_revenue.drop_duplicates(subset=['Date'], keep='first')
    gme_revenue = gme_revenue.sort_values('Date')
    gme_revenue = gme_revenue.reset_index(drop=True)

print("GameStop Revenue Data - Last 5 rows:")
print(gme_revenue.tail())


GameStop Revenue Data - Last 5 rows:
          Date  Revenue
79  2024-10-31    860.0
80        2025   3823.0
81  2025-01-31   1283.0
82  2025-04-30    732.0
83  2025-07-31    972.0


## Question 5: Plot Tesla Stock Graph

Use the make_graph function to graph the Tesla Stock Data, also provide a title for the graph.


In [ ]:
def make_graph(stock_data, revenue_data, stock):
    try:
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                            subplot_titles=("Historical Share Price", "Historical Revenue"), 
                            vertical_spacing = .3)
        
        stock_data_specific = stock_data[stock_data['Date'] < '2021-01-01']
        
        fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific['Date'], infer_datetime_format=True), 
                                 y=stock_data_specific['Close'].astype("float"), 
                                 name="Share Price"), row=1, col=1)
        
        if not revenue_data.empty:
            try:
                revenue_data['Date'] = pd.to_datetime(revenue_data['Date'], format='mixed', dayfirst=False)
            except:
                try:
                    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'], format='ISO8601')
                except:
                    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'], infer_datetime_format=True)
            
            revenue_data_specific = revenue_data[revenue_data['Date'] < '2021-01-01']
            
            if not revenue_data_specific.empty:
                revenue_data_specific = revenue_data_specific.drop_duplicates(subset=['Date'], keep='first')
                revenue_data_specific = revenue_data_specific.sort_values('Date')
                
                fig.add_trace(go.Scatter(x=revenue_data_specific['Date'], 
                                         y=revenue_data_specific['Revenue'].astype("float"), 
                                         name="Revenue"), row=2, col=1)
        
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_xaxes(title_text="Date", row=2, col=1)
        fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
        fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
        
        fig.update_layout(showlegend=False,
                          height=900,
                          title_text=f"{stock} Stock Analysis")
        
        fig.show()
        
    except Exception as e:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        stock_data_specific = stock_data[stock_data['Date'] < '2021-01-01']
        ax1.plot(pd.to_datetime(stock_data_specific['Date']), 
                stock_data_specific['Close'], 
                label='Share Price', color='blue')
        ax1.set_title(f'{stock} Historical Share Price')
        ax1.set_ylabel('Price ($US)')
        ax1.legend()
        ax1.grid(True)
        
        if not revenue_data.empty:
            try:
                revenue_data['Date'] = pd.to_datetime(revenue_data['Date'], format='mixed', dayfirst=False)
            except:
                try:
                    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'], format='ISO8601')
                except:
                    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'], infer_datetime_format=True)
            
            revenue_data_specific = revenue_data[revenue_data['Date'] < '2021-01-01']
            
            if not revenue_data_specific.empty:
                revenue_data_specific = revenue_data_specific.drop_duplicates(subset=['Date'], keep='first')
                revenue_data_specific = revenue_data_specific.sort_values('Date')
                
                ax2.plot(revenue_data_specific['Date'], 
                        revenue_data_specific['Revenue'], 
                        label='Revenue', color='green')
                ax2.set_title(f'{stock} Historical Revenue')
            else:
                ax2.text(0.5, 0.5, f'No revenue data available for {stock}', 
                        ha='center', va='center', transform=ax2.transAxes)
                ax2.set_title(f'{stock} Historical Revenue (No Data)')
        else:
            ax2.text(0.5, 0.5, f'No revenue data available for {stock}', 
                    ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title(f'{stock} Historical Revenue (No Data)')
        
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Revenue ($US Millions)')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plt.show()

make_graph(tesla_data, tesla_revenue, 'Tesla')


Creating Tesla Stock Graph...
Plotting 58 revenue data points for Tesla


Successfully created Tesla graph using Plotly


## Question 6: Plot GameStop Stock Graph

Use the make_graph function to graph the GameStop Stock Data, also provide a title for the graph.


In [ ]:
make_graph(gme_data, gme_revenue, 'GameStop')


Creating GameStop Stock Graph...
Plotting 60 revenue data points for GameStop


Successfully created GameStop graph using Plotly
